<a href="https://colab.research.google.com/github/jcquimpo/GoogleColab_NBs/blob/main/IS392_Proj_ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

In [ ]:
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

test = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


## Preprocessing Text - V1
- Preprocessing for Logistic Regression and Linear SVC


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from bs4 import BeautifulSoup
import string
from sklearn import model_selection, metrics
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import csv
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

* Import the missing imports for the functions

In [ ]:
# import seaborn as sns
# import plotly.express as px
# from wordcloud import WordCloud

In [ ]:
#Remove Hashtags
train['review'].replace( { r"#(\w+)" : '' }, inplace= True, regex = True)
#Remove Mention
train['review'].replace( { r"@(\w+)" : '' }, inplace= True, regex = True)
#Remove URL
train['review'].astype(str).replace( { r"http\S+" : '' }, inplace= True, regex = True)
train['review']=train['review'].str.lower()

In [ ]:
y = train['sentiment']
x = train['review']

In [ ]:
# def data_processing(text):
#     text = text.lower()
#     text = re.sub('<br />', '', text)
#     text = re.sub(r"https\S+|www\S+|http\S+", '', text, flags = re.MULTILINE)
#     text = re.sub(r'\@w+|\#', '', text)
#     text = re.sub(r'[^\w\s]', '', text)
#     text_tokens = word_tokenize(text)
#     filtered_text = [w for w in text_tokens if not w in stop_words]
#     return " ".join(filtered_text)
# df.review = df['review'].apply(data_processing)

In [ ]:
vectorizer1 = TfidfVectorizer(max_features=1500)
x1 = vectorizer1.fit_transform(x)
feature_names1 = vectorizer1.get_feature_names_out()
denselist1 = x1.todense().tolist()
train = pd.DataFrame(denselist1, columns=feature_names1)

# splitting the training and testing part from the data
X_temp, X_test, y_temp, y_test = train_test_split(train, y, test_size=0.1, random_state=0)

## Preprocessing Text - V2
- Embedding and Cleaning for BERT  

### This form of preprocessing was to be done for the BERT model, but the BERT model does not install the dependicices properly
  - don't need to do this preprocessing process

In [ ]:
# def cleanText(text):

#     text = re.sub(r'<.*?>', ' ', text)
#     text = re.sub(r"won't", "will not", text)
#     text = re.sub(r"can't", "can not", text)
#     text = re.sub(r"n't", " not", text)
#     text = re.sub(r"'ve", " have", text)
#     text = re.sub(r"'ll", " will", text)
#     text = re.sub(r"'re", " are", text)

#     # if embedding is not 'BERT':
#     #     text = re.sub(r"[0-9]+", ' ', text)
#     #     text = re.sub(r"-", ' ', text)


#     text = text.strip().lower()

#     # if embedding is 'WORD2VEC_NO_STOP':
#     #     # Remove Stop words
#     #     default_stop_words = set(stopwords.words('english'))
#     #     default_stop_words.difference_update({'no', 'not', 'nor', 'too', 'any'})
#     #     stop_words = default_stop_words.union({"'m", "n't", "'d", "'re", "'s",
#     #                                            'would','must',"'ve","'ll",'may'})

#     #     word_list = word_tokenize(text)
#     #     filtered_list = [w for w in word_list if not w in stop_words]
#     #     text = ' '.join(filtered_list)

#     # if embedding is not 'BERT':
#     #     # Remove other contractions
#     #     text = re.sub(r"'", ' ', text)

#     # Replace punctuations with space
#     if embedding is 'BERT': # save ! ? . for end of the sentence detection [,/():;']
#         filters='"#$%&*+<=>@[\]^_`{|}~\t\n'
#         text = re.sub(r'\!+', '!', text)
#         text = re.sub(r'\?+', '?', text)
#     else:
#         filters='!"\'#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n'
#     translate_dict = dict((i, " ") for i in filters)
#     translate_map = str.maketrans(translate_dict)
#     text = text.translate(translate_map)

#     if embedding is 'BERT':
#         text = re.sub(r'', ' ', text)

#     # if embedding is not 'BERT':
#     #     text = ' '.join([w for w in text.split() if len(w)>1])

#     # Replace multiple space with one space
#     text = re.sub(' +', ' ', text)

#     text = ''.join(text)

#     return text

## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
accuracy = {'TF-IDF':[], 'BoW': []}

In [ ]:
#TF-IDF
# regressor_LR_tf = LogisticRegression(C=1,penalty='l2',solver='liblinear')
regressor_LR_tf = LogisticRegression(C=1,penalty='l2',solver='newton-cg')
regressor_LR_tf.fit(X_temp, y_temp)
y_predict_LR_tf = regressor_LR_tf.predict(X_test)
a=(regressor_LR_tf.score(X_test, y_test))
accuracy['TF-IDF'].append(a)

print(metrics.confusion_matrix(y_test, y_predict_LR_tf))
print(metrics.classification_report(y_test, y_predict_LR_tf))
print(metrics.accuracy_score(y_test, y_predict_LR_tf))

[[1105  178]
 [ 136 1081]]
              precision    recall  f1-score   support

           0       0.89      0.86      0.88      1283
           1       0.86      0.89      0.87      1217

    accuracy                           0.87      2500
   macro avg       0.87      0.87      0.87      2500
weighted avg       0.87      0.87      0.87      2500

0.8744


## Linear SVC
- https://github.com/roshancyriacmathew/Machine-Learning-on-Movie-Reviews-IMDB-Dataset---50k-reviews-/blob/main/Sentiment%20analysis%20on%20IMDB%20dataset%20_%20ML%20Live.ipynb

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
svc = LinearSVC()
svc.fit(X_temp, y_temp)
svc_pred = svc.predict(X_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("Test accuracy: {:.2f}%".format(svc_acc*100))

print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

Test accuracy: 86.84%
[[1100  183]
 [ 146 1071]]


              precision    recall  f1-score   support

           0       0.88      0.86      0.87      1283
           1       0.85      0.88      0.87      1217

    accuracy                           0.87      2500
   macro avg       0.87      0.87      0.87      2500
weighted avg       0.87      0.87      0.87      2500



## BERT
- Sentence tokenization
- https://github.com/hmohebbi/SentimentAnalysis/blob/master/main.ipynb
- did not work because ModuleNotFoundError
  - could not install properly --> unsure why

In [ ]:
# pip install bert-embeddings

In [ ]:
# from bert_embeddings import BertEmbedding
# from sentence_transformers import SentenceTransformer

In [ ]:
# # Cleaning before BERT
# embedding = 'BERT'
# # for BERT(s)
# train['clean_text_bert'] = train['review'].apply(cleanText)
# test['clean_text_bert'] = test['review'].apply(cleanText)

# print("clean_text_bert:")
# print(train_data.iloc[3]['clean_text_bert'], end="\n\n")

In [ ]:
# bert_transformers = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# bert_version = 'SENTENCE'
# bert_sentence_training_features = train_data['clean_text_bert'].apply(embeddToBERT)
# bert_sentence_test_features = test_data['clean_text_bert'].apply(embeddToBERT)


# feature = [x for x in bert_sentence_training_features.transpose()]
# bert_sentence_training_features = np.asarray(feature)

# feature = [x for x in bert_sentence_test_features.transpose()]
# bert_sentence_test_features = np.asarray(feature)

# print(bert_sentence_training_features.shape)

## CNN Model
  - https://github.com/abdel/imdb-sentiment-analysis/blob/master/notebooks/CNN.ipynb

In [ ]:

# from sklearn.metrics import confusion_matrix, classification_report, f1_score
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation
# from keras.layers import Embedding, SpatialDropout1D
# from keras.layers import LSTM
# from keras.layers import Conv1D, GlobalMaxPooling1D
# from keras.layers import Flatten
# from keras.datasets import imdb
# from keras.utils import plot_model
# from keras.utils.vis_utils import model_to_dot

In [ ]:
# # Embedding
# embedding_size = 50
# max_features = 5000
# maxlen = 400

# # Convolution
# kernel_size = 3
# pool_size = 4
# filters = 250

# # Dense
# hidden_dims = 250

# # Training
# batch_size = 64
# epochs = 4

In [ ]:
# model = Sequential()

# # we start off with an efficient embedding layer which maps
# # our vocab indices into embedding_dims dimensions
# model.add(Embedding(max_features,
#                     embedding_size,
#                     input_length=maxlen))
# model.add(Dropout(0.2))

# model.add(Conv1D(filters,
#                 kernel_size,
#                 padding='valid',
#               activation='relu',
#                 strides=1))
# model.add(GlobalMaxPooling1D())

# # We add a vanilla hidden layer:
# model.add(Dense(hidden_dims))
# model.add(Dropout(0.2))
# model.add(Activation('relu'))

# # We project onto a single unit output layer, and squash it with a sigmoid:
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# # Train the model
# model.fit(X_temp, y_temp,
#          batch_size=batch_size,
#          epochs=epochs,
#          validation_data=(X_test, y_test),
#          verbose=1)

## DistilBert (?)

In [ ]:
import tensorflow as tf

# num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
# print("Num GPUs Available: ", num_gpus_available)
# assert num_gpus_available > 0

In [ ]:
train_DB = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

test_DB = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
train_DB.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
sentiments = train_DB['sentiment'].values.tolist()
reviews = train_DB['review'].values.tolist()

In [ ]:
training_sentences_DB, validation_sentences_DB, training_labels_DB, validation_labels_DB = train_test_split(reviews, sentiments, test_size=.2)

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

tokenizer_DB = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer_DB([training_sentences_DB[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 1000, 2023, 2003, 5262, 3243, 1996, 4326, 3185, 1012, 1012, 1012, 2034, 1010, 2057, 2031, 2019, 4654, 1011, 1057, 1012, 1055, 1012, 1011, 25055, 2667, 2000, 2735, 3364, 1006, 2030, 2242, 1007, 1010, 1998, 2023, 3849, 2000, 2022, 1996, 2069, 2535, 2002, 2412, 2288, 1006, 2008, 1045, 2113, 1997, 4312, 1007, 1011, 1011, 1998, 2005, 2204, 3114, 1012, 2096, 2002, 2515, 4139, 2125, 1996, 2535, 2092, 2438, 2000, 2562, 2070, 3037, 1010, 2009, 2003, 1037, 2738, 20857, 1998, 4257, 2836, 1012, 2117, 1010, 2057, 2031, 1996, 5409, 2412, 2614, 3896, 2412, 2109, 1999, 1037, 3185, 999, 999, 999, 1045, 1005, 1049, 2025, 12489, 1012, 2023, 2894, 3084, 1996, 3185, 5186, 29257, 1010, 2021, 1999, 2008, 15703, 2126, 1012, 2002, 5369, 1998, 2353, 1010, 2096, 2057, 2031, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
train_encodings = tokenizer_DB(training_sentences_DB,
                            truncation=True,
                            padding=True)

val_encodings = tokenizer_DB(validation_sentences_DB,
                            truncation=True,
                            padding=True)

In [ ]:
train_dataset_DB = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels_DB
))

val_dataset_DB = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels_DB
))

In [ ]:
model_DB = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                              num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model_DB.compile(optimizer=optimizer, loss=model_DB.hf_compute_loss, metrics=['accuracy'])
model_DB.fit(train_dataset_DB.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset_DB.shuffle(100).batch(16))

Epoch 1/2
  25/1250 [..............................] - ETA: 13:36:38 - loss: 0.6681 - accuracy: 0.6000

In [ ]:
# test_sentence = "DoiT is a great company"
# # replace to test_sentence_sarcasm variable, if you want to test sarcasm
# predict_input = tokenizer.encode(test_sentence,
#                                  truncation=True,
#                                  padding=True,
#                                  return_tensors="tf")
# tf_output = model_DB.predict(predict_input)[0]
# tf_prediction = tf.nn.softmax(tf_output, axis=1)
# labels = ['Negative','Positive']
# label = tf.argmax(tf_prediction, axis=1)
# label = label.numpy()
# print(labels[label[0]])

## Sentiment Tokenization
- https://www.kaggle.com/code/kiernanmcguigan/sentiment-bag-of-words-tokenization


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import callbacks, models, layers
import matplotlib.pyplot as plt

# tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
BASE = '/kaggle/input/word2vec-nlp-tutorial'
MAX_WORDS = 25_000

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# train = pd.read_csv(os.path.join(BASE,'labeledTrainData.tsv.zip'), header=0, delimiter="\t", quoting=3)
# test = pd.read_csv(os.path.join(BASE,'testData.tsv.zip'), header=0, delimiter="\t", quoting=3)

train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

test = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [ ]:
train.head
test.head

In [ ]:
print(train["sentiment"][0])
train["review"][0]

stop_words = stopwords.words("english")
def clean(review):
    clean_html = BeautifulSoup(review).get_text()
    clean_non_letters = re.sub("[^a-zA-Z]", " ", clean_html)
    cleaned_lowercase = clean_non_letters.lower()
    words = cleaned_lowercase.split()
    cleaned_words = [w for w in words if w not in stop_words]
    return " ".join(cleaned_words)

train["cleaned_review"] = train["review"].apply(clean)
# train

1


<ipython-input-28-94bd4fddeb5b>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_html = BeautifulSoup(review).get_text()


In [ ]:
# with no constraints there are 74_066 words in the training set
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train.cleaned_review)
total_words = len(tokenizer.word_index) + 1
# total_words

In [ ]:
sequences = tokenizer.texts_to_sequences(train.cleaned_review)
max_sequence_len = max([len(x) for x in sequences])
padded_sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))
labels = np.array(train.sentiment)
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(20000, 1322) (20000,) (5000, 1322) (5000,)


In [ ]:
def to_dataset(data, labels):
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.cache().shuffle(X_train.shape[0] + 1).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset
train_ds = to_dataset(X_train, y_train)
val_ds = to_dataset(X_val, y_val)

In [ ]:
LSTM_SIZE = 8
def bi_lstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(total_words, 16, input_length=max_sequence_len - 1))
    model.add(layers.Bidirectional(layers.LSTM(LSTM_SIZE)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model, f'bidirectional_lstm_{LSTM_SIZE}'

def lstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(total_words, 4, input_length=max_sequence_len - 1))
    model.add(layers.LSTM(LSTM_SIZE))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model, f'lstm_{LSTM_SIZE}'

In [ ]:
def tokenizer_train(model, name):
    reducer = callbacks.ReduceLROnPlateau(monior='val_loss', factor=0.5, patience=3, mode='min', cooldown=1)
    stopper = callbacks.EarlyStopping(monitor='val_loss', patience=6, mode='min', restore_best_weights=True)
    hist = model.fit(train_ds,
              epochs=100,
              verbose=1,
              callbacks=[stopper, reducer],
              validation_data=val_ds)
    results = model.evaluate(val_ds)
    # model.save(f'/kaggle/working/{name}')
    print(f"results: {results}, type: {type(results)}")
    return hist

In [ ]:
model, name = lstm_model()
model.summary()

In [ ]:
hist = tokenizer_train(model, name)

Epoch 1/100


ValueError: ignored

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8,8), tight_layout=True)

axs[0].plot(hist.history['loss'])
axs[0].plot(hist.history['val_loss'])
axs[0].set_title('binary_crossentropy Loss')
axs[0].set_ylabel('Loss')
axs[0].set_xlabel('Epoch')
axs[0].legend(['train', 'val'], loc='upper right')

axs[1].plot(hist.history['binary_accuracy'])
axs[1].plot(hist.history['val_binary_accuracy'])
axs[1].set_title('binary_accuracy Metric')
axs[1].set_ylabel('Error')
axs[1].set_xlabel('Epoch')
axs[1].legend(['train', 'val'], loc='upper left')

axs[2].plot(hist.history['lr'])
axs[2].set_title('Learining Rate')
axs[2].set_ylabel('LR')
axs[2].set_xlabel('Epoch')
plt.savefig(f'/kaggle/working/{name}_graphs.png')
plt.show()

In [ ]:
test["cleaned_review"] = test["review"].apply(clean)
test

<ipython-input-28-94bd4fddeb5b>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_html = BeautifulSoup(review).get_text()


KeyboardInterrupt: ignored

In [ ]:
sequences = tokenizer.texts_to_sequences(test.cleaned_review)
test_sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))
print(test_sequences.shape)

predictions = model.predict(test_sequences).flatten()
predictions.shape

## LSTM with Keras
  - https://www.kaggle.com/code/nichen301/movie-reviews-lstm-with-keras

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

# import os
# print(os.listdir("../input"))

In [ ]:
train_data = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
# train_imdb = train_imdb[train_imdb.label != 'unsup']
# train_imdb["sentiment"] = train_imdb.label.map({"neg": 0, "pos": 1})
# train_imdb.drop(["type", "file", "label"], axis=1, inplace=True)
# train_enhanced = pd.concat([train, train_imdb], ignore_index=True)

# X_train = train_enhanced.review
# y_train = train_enhanced.sentiment

X_train = train_data.review
y_train = train_data.sentiment

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download()

stopwords_eng = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub("[^a-z]", " ", text)
    words = [word for word in text.split() if word not in stopwords_eng]
    text = " ".join(words)
    return text

X_train = X_train.map(clean_text)
X_train.head()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


0    stuff going moment mj started listening music ...
1    classic war worlds timothy hines entertaining ...
2    film starts manager nicholas bell giving welco...
3    must assumed praised film greatest filmed oper...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

In [ ]:
from keras.preprocessing.text import Tokenizer
num_words = 5000
tokenizer = Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(X_train)

X_train_seq = pd.Series(tokenizer.texts_to_sequences(X_train))
X_train_seq.head()

0    [405, 71, 421, 508, 2460, 116, 55, 875, 517, 1...
1    [233, 204, 3051, 3567, 318, 3, 406, 154, 20, 6...
2    [3, 383, 2821, 4397, 3731, 599, 2185, 519, 431...
3    [101, 4830, 3, 683, 663, 1260, 43, 213, 1039, ...
4    [3366, 4135, 2039, 1570, 747, 495, 697, 502, 9...
dtype: object

In [ ]:
X_train_len = X_train_seq.map(lambda ls: len(ls))
X_train_len.describe()

count    25000.000000
mean       102.165360
std         74.185126
min          3.000000
25%         56.000000
50%         77.000000
75%        125.000000
max        941.000000
dtype: float64

In [ ]:
# from keras_preprocessing.sequence import pad_sequences
# this is the correct import
# https://stackoverflow.com/questions/72326025/cannot-import-name-pad-sequences-from-keras-preprocessing-sequence --> imaspol
from keras.utils import pad_sequences

X_train_pad = pad_sequences(X_train_seq, maxlen=256)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout

model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=64))
model.add(LSTM(32, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          12416     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                        

In [ ]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 5
validation_split = 0.045
model.fit(x=X_train_pad, y=y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/5
747/747 [==============================] - 112s 144ms/step - loss: 0.0974 - accuracy: 0.9702 - val_loss: 0.3853 - val_accuracy: 0.8747
Epoch 2/5
747/747 [==============================] - 106s 141ms/step - loss: 0.0607 - accuracy: 0.9825 - val_loss: 0.4839 - val_accuracy: 0.8773
Epoch 3/5
747/747 [==============================] - 106s 142ms/step - loss: 0.0481 - accuracy: 0.9857 - val_loss: 0.5201 - val_accuracy: 0.8684
Epoch 4/5
747/747 [==============================] - 107s 144ms/step - loss: 0.0368 - accuracy: 0.9889 - val_loss: 0.5576 - val_accuracy: 0.8658
Epoch 5/5
747/747 [==============================] - 110s 147ms/step - loss: 0.0273 - accuracy: 0.9915 - val_loss: 0.6017 - val_accuracy: 0.8684


In [ ]:
test_data = pd.read_csv("testData.tsv", header=0, \
                    delimiter="\t", quoting=3)

X_test = test_data.review.map(clean_text)
X_test.head()

0    naturally film main themes mortality nostalgia...
1    movie disaster within disaster film full great...
2    movie kids saw tonight child loved one point k...
3    afraid dark left impression several different ...
4    accurate depiction small time mob life filmed ...
Name: review, dtype: object

In [ ]:
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=512)

In [ ]:
pred = model.predict(x=X_test_pad)
y_pred = (pred >= 0.5) * 1
results = pd.DataFrame({"id": test_data.id.index, "sentiment": y_pred.flatten()})


782/782 [==============================] - 49s 63ms/step


In [ ]:
results['id'] = results["id"].astype("str")
results.to_csv("submission.csv", index=False)

results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB
